In [ ]:
import plotly.express as px
import numpy as np
import duckdb
from tqdm import tqdm

In [ ]:
import sys
sys.path.append("/home/ubuntu/sky_workdir/encoding-schemes")

from encoding_schemes import get_deterministic_adherence_fn

In [ ]:
import ray

ray.init()

In [ ]:
import os
import psycopg2
import json

conn_string = os.environ["SUPABASE_CONNECTION_URL"]

conn = psycopg2.connect(conn_string)

import pandas as pd

In [ ]:
sel_str = """
-- prompted no sft decode
(
    (data->'experiment_tags'->'numina_math_cot_rerun')::BOOL
    AND (NOT (data->'force_overwrite')::BOOL OR data->'force_overwrite' IS NULL)
    AND (data->'experiment_name')::TEXT LIKE '%prompteddecode%'
)
"""

df = pd.read_sql(f"""
SELECT * FROM public.encoding_schemes 
    WHERE 

{sel_str}


ORDER BY created_at DESC
""", conn)

df.head()

In [ ]:
root_dir = "/home/ubuntu/sky_workdir/encoding-schemes/output"

In [ ]:
l_examples = df.to_dict('records')

# Code

In [ ]:

def bootstrap_ci(data, statistic=np.mean, alpha=0.05, n_boot=10_000, random_state=None):
    """
    Returns (point_estimate, low_CI, high_CI) for given 1D data.
    Works with bool, int, or float data.
    """
    x = np.asarray(data).astype(float)  # ensure numeric
    x = x[~np.isnan(x)]
    if len(x) == 0:
        raise ValueError("No valid data for bootstrapping.")

    rng = np.random.default_rng(random_state)
    n = len(x)

    # Draw bootstrap samples
    idx = rng.integers(0, n, size=(n_boot, n))
    samples = x[idx]

    # Apply statistic row-wise
    stats = np.apply_along_axis(statistic, 1, samples)

    point = statistic(x)
    lo = np.percentile(stats, 100 * (alpha / 2))
    hi = np.percentile(stats, 100 * (1 - alpha / 2))
    return point, lo, hi

In [ ]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")


@ray.remote
def count_tokens_from_messages(s):
    try:
        return len(encoding.encode(s, disallowed_special=()))
    except ValueError as e:
        print(e)
        return 0


In [ ]:
@ray.remote(num_cpus=2, memory=32 * 1024 * 1024 * 1024)
def compute_translation_token_count(example, df_data):
    sys.path.append("/home/ubuntu/sky_workdir/encoding-schemes")

    from orchestration.experiment_meta_saver import compute_experiment_hash
    from utils.io_utils import read_large_parquet

    from transformers import AutoTokenizer

    model = example["data"]["experiment_params"]["model"]
    if "gpt" in model or "claude" in model:
        print(f"Overriding tokenizer for {model} with gpt-oss 120b tokenizer because it was detected as a GPT/Claude model!")
        model = "openai/gpt-oss-120b"

    tokenizer = AutoTokenizer.from_pretrained(model)

    return df_data['reference_solution'].map(lambda x: len(tokenizer.encode(x)))


In [ ]:
from tqdm import tqdm

In [ ]:
def compute_ci_cols(example, df, col_name, transformation_fn):
    s_transformed = transformation_fn(df[col_name])
    if np.isscalar(s_transformed) and np.isnan(s_transformed):
        print(f"Warning: {col_name} was all NaN, ignoring!")
        return
    mid, lo, hi = bootstrap_ci(s_transformed)
    example[col_name] = mid
    example[f'{col_name}_low_ci'] = mid - lo
    example[f'{col_name}_hi_ci'] = hi - mid

In [ ]:
def compute_multi_row_transformation(df, row_transform_fn, col_name, agg_fn):
    df[col_name] = df.apply(row_transform_fn, axis=1)

In [ ]:

def _score_rollouts(rollouts):
    # rollouts: list/array of rollout sequences; may also be None/np.nan/scalar
    if rollouts is None or (isinstance(rollouts, float) and np.isnan(rollouts)):
        return np.nan

    vals = []
    for r in rollouts:
        # skip None/NaN
        if r is None or (isinstance(r, float) and np.isnan(r)):
            continue
        vals.append(-np.nansum(r))
    return np.nanmean(vals) if len(vals) else np.nan


def patch_gpt_api_log_loss(example):
    experiment_hash = example['experiment_hash']

    translation_loss = os.path.join(root_dir, experiment_hash, "data", f"validation_reverse_translation_math500_meta.json")
    with open(translation_loss, "r") as fp:
        example["backtranslation_gt_logprobs"] = translation_loss["valid_loss"]

    # need to be validation loss on 512k...
    validation_loss = os.path.join(root_dir, experiment_hash, "data", f"validation_reverse_translation_math500_meta.json")
    with open(validation_loss, "r") as fp:
        example["backtranslation_gt_logprobs"] = translation_loss["valid_loss"]


@ray.remote
def process_single_example(example):
    experiment_hash = example['experiment_hash']
    
    target_path = os.path.join(root_dir, example['experiment_hash'], "data", "joined_output.parquet")
    if not os.path.exists(target_path):
        print(f"!!!!!! {target_path} missing !!!!!!!")
        return example
    
    df_data = pd.read_parquet(target_path)

    d_col_to_transform = {
        'cot_gt_logprobs' : lambda s: np.nansum(s.map(_score_rollouts)),
        'generated_cot_is_correct' : np.mean,  # was np.mean
        'backtranslation_gt_logprobs' : lambda s: np.nanmean(s.map(_score_rollouts)),
        'backtranslation_bleu_scores' : np.mean,  # was np.mean
        'generated_cot_adhered_encoding_style': np.mean  # was np.mean
    }
    for col, fn in d_col_to_transform.items():
        if col not in df_data:
            print(col)
            print(df_data.head())
            print(example)
            raise Exception(str(col) + "\n" + str(df_data.head()) + "\n" + str(example))

        compute_ci_cols(example, df_data, col, fn)

    df_data["num_tokens_translation_output"] = ray.get(compute_translation_token_count.remote(example, df_data))

    d_and_cols = {
        'adherent_and_correct': (
            lambda r: np.nanmean( \
                np.array(r['generated_cot_is_correct']).astype(bool) & \
                np.array(r['generated_cot_adhered_encoding_style']).astype(bool) \
            ),
            np.nanmean
        ),
        'total_translation_loss': (
            lambda r: np.nanmean( \
                np.array(r['num_tokens_translation_output']) * \
                np.array(np.nanmean(_score_rollouts(r['backtranslation_gt_logprobs'])), dtype=np.float64) \
            ),
            np.nanmean
        ),
    }
    for col, (transform_fn, agg_fn) in d_and_cols.items():
        compute_multi_row_transformation(df_data, transform_fn, col, agg_fn)
        if df_data[col].isna().sum() != len(df_data):
            compute_ci_cols(example, df_data, col, lambda x: x)

    if "gpt" in example["data"]["experiment_params"]["model"] and "use_api_sft_model_for_sampling" in example["data"]["experiment_params"]:
        with open(os.path.join(root_dir, example['experiment_hash'], "data", f"validation_reverse_translation_math500_meta.json"), "r") as fp:
            d_model_meta = json.load(fp)

        example["backtranslation_gt_logprobs"] = d_model_meta["valid_loss"]
        example["total_translation_loss"] = d_model_meta["valid_loss"] * np.nanmean(df_data["num_tokens_translation_output"])
        example["total_translation_loss_low_ci"] = 0.0
        example["total_translation_loss_hi_ci"] = 0.0

    pretraining_prevalence_path = os.path.join('/home/ubuntu/sky_workdir/encoding-schemes', 'output', experiment_hash, 'data', 'num_pretraining_4grams_redpajama.json')
    if os.path.exists(pretraining_prevalence_path):
        with open(pretraining_prevalence_path, "r") as fp:
            d_pretraining_prevalence = json.load(fp)

        example["pretraining_prevalence"] = d_pretraining_prevalence["num_occurrences"]

    for col in df_data.columns:
        example[f"{col}_df"] = df_data[col]

    return example


l_new_examples = [None for _ in range(len(l_examples))]

for i, example in tqdm(enumerate(l_examples)):
    # l_examples[i] = process_single_example(example)
    l_new_examples[i] = process_single_example.remote(example)

for i, example in tqdm(enumerate(l_new_examples)):
    try:
        l_new_examples[i] = ray.get(example)
    except ray.exceptions.RayTaskError as e:
        l_new_examples[i] = l_examples[i]
        print(e)

l_examples = l_new_examples

In [ ]:
def humanize_number(num: float) -> str:
    """
    Converts a number into a human-readable string with k, M, or B suffixes.
    
    Args:
        num (float): The number to format.
    
    Returns:
        str: Human-readable string representation.
    """
    if num >= 1_000_000_000:
        return f"{num / 1_000_000_000:.1f}B"
    elif num >= 1_000_000:
        return f"{num / 1_000_000:.1f}M"
    elif num >= 1_000:
        return f"{num / 1_000:.1f}k"
    else:
        return str(num)

In [ ]:
import re

def parse_params(model):
    if 'gpt' in model:
        if 'nano' in model:
            return 0
        elif 'mini' in model:
            return 1
        else:
            return 2


    if 'claude' in model:
        if 'haiku' in model:
            return 3
        elif '3-opus' in model:
            return 4
        elif '3-5' in model:
            return 5
        elif 'sonnet-4' in model:
            return 6
        else:
            return 7
    
    return int(re.search("([0-9]+)B", model).group(1))

In [ ]:
df_viz = pd.DataFrame(l_examples)

orig_len = len(df_viz)

# df_viz = df_viz[df_viz['cot_gt_logprobs'].notna()]

new_len = len(df_viz)
if orig_len != new_len:
    print(f"Lost {orig_len - new_len} examples from na logprobs")

df_viz['encoding_scheme'] = df_viz['data'].map(lambda x: x['experiment_params']['encoding_scheme'])
df_viz['model'] = df_viz['data'].map(lambda x: x['experiment_params']['model'])

try:
    df_viz['model_size'] = df_viz['model'].map(parse_params)
except Exception as e:
    print(e)
df_viz['input_type'] = df_viz['data'].map(lambda x: "_".join(x['experiment_name'].split("_")[:2]))

df_viz['n_few_shot_examples'] = df_viz['data'].map(lambda x: x['experiment_params'].get('n_few_shot_examples', None))

df_viz['Adherence Calculation Method'] = df_viz['encoding_scheme'].map(lambda x: get_deterministic_adherence_fn(x, None) is not None).map({ True: 'deterministic', False: 'Sonnet 4 judge'})

try:
    df_viz['total_train_tok'] = df_viz['n_total_train_tok'].map(humanize_number)
except Exception as e:
    print(e)


In [ ]:
filter_set = ['mathcot_prompteddecode']

In [ ]:
df_viz_tmp = df_viz[df_viz['input_type'].isin(filter_set)]
df_viz_tmp = df_viz_tmp[df_viz_tmp['model'] != 'Qwen/Qwen2.5-7B']

df_viz_tmp = df_viz_tmp.sort_values([
    'model_size',
    'adherent_and_correct'
])

df_viz_tmp = df_viz_tmp.astype({'n_few_shot_examples': str})

df_viz_tmp['encoding_scheme'] = df_viz_tmp['encoding_scheme'].map(lambda s: s.split("speaking_")[-1])

# Paper plots

In [ ]:
d_mapping = {
    "baseline": ["zero_shot", "identity"],
    "letter mutation": [
        "reverse_letters_in_each_word",
        "swap_even_odd_letters_in_each_word",
        "reverse_fibonacci_indices_in_each_word",
        "letter_to_word_with_dot",
        "dot_between_chars",
        "space_between_chars",
    ],
    "language deletion": ["remove_all_verbs", "remove_all_nouns"],
    "language translation": [
        "French","Chinese","Korean","Russian","Arabic","Adyghe",
        "Morse_code","Python","enterprise_Java",
    ],
    "algorithmic cipher": [
        "rot13_cipher","base64_cipher","base64_2x_cipher",
        "base64_3x_cipher","caesar_cipher","gzip_to_base64_encoded",
    ],
    "themed reasoning": [
        "paraphrase_naive",
        "pirate_speak",
        "leet_speak",
        "yoda_speak",
        "shakespearean_text",
    ],
    "extraneous content": [
        "insert_tweet",
        "python_snippet_comment",
        "croissant_news_article",
        "math_textbook_article",
        "five_emojis",
    ],
    "delete inf.": [
        "replace_math_content_with_black_box"
    ]
}

l_themed_encodings = [
    "remove_all_verbs",
    "paraphrase_naive",
    "pirate_speak",
    "leet_speak",
    "yoda_speak",
    "shakespearean_text",
    "insert_tweet",
    "python_snippet_comment",
    "croissant_news_article",
    "math_textbook_article",
    "five_emojis",
    "replace_math_content_with_black_box",
    "reverse_letters_in_each_word_no_math_expressions",
    "reverse_letters_in_each_word_only_math_expressions"
]

l_ignore_languages = [
    "Russian",
    "Chinese",
    # "Korean"
]

In [ ]:
import plotly.express as px
import plotly.colors as pc

def sample_colorscale(colorscale_name: str, n: int):
    """
    Sample N equally spaced hex colors from a Plotly continuous colorscale.

    Parameters
    ----------
    colorscale_name : str
        Name of a Plotly built-in continuous colorscale (e.g., "Viridis", "Blues").
    n : int
        Number of samples to return.

    Returns
    -------
    list of str
        List of hex color strings.
    """
    if n < 1:
        raise ValueError("n must be >= 1")
    colorscale = px.colors.get_colorscale(colorscale_name)
    # equally spaced points in [0,1]
    vals = [i/(n-1) if n > 1 else 0.5 for i in range(n)]
    return [pc.sample_colorscale(colorscale, v)[0] for v in vals]

l_gpt_gradient = sample_colorscale("Emrld", 4 + 1)
d_gpt_gradient = {
    model : color
    for model, color in zip(
        ['gpt-4.1-nano-2025-04-14', 'gpt-4.1-mini-2025-04-14', 'gpt-4.1-2025-04-14', 'gpt-5-chat-latest'],
        l_gpt_gradient[:-1]
    )
}

l_claude_gradient = sample_colorscale("Purpor", 3 + 1)
d_claude_gradient = {
    model : color
    for model, color in zip(
        ['claude-3-opus-20240229', 'claude-3-5-sonnet-20241022', 'claude-sonnet-4-20250514'],
        l_claude_gradient[1:]
    )
}

l_qwen_gradient = sample_colorscale("Oryel", 3 + 1)
d_qwen_gradient = {
    model : color
    for model, color in zip(
        ['Qwen2.5-3B-Instruct', 'Qwen2.5-7B-Instruct', 'Qwen2.5-14B-Instruct'],
        l_qwen_gradient[1:]
    )
}

In [ ]:
l_star_encodings = [
    "reverse_letters_in_each_word",
    "rot13_cipher",
    "caesar_cipher", # TODO check
 'reverse_fibonacci_indices_in_each_word', # TODO check
 'swap_even_odd_letters_in_each_word', # TODO check
    "Morse_code"
]

In [ ]:
l_fixed_encoding_ordering = [
    # GPT
 'identity',
'dot between chars',
 'rot13 cipher',
 'Korean',
 'reverse letters in each word',
 'letter to word with dot',
     'base64 cipher',
 # 'remove all verbs',

    # placeholder for an empty space
    '',

    # Qwen/Claude
'space between chars',
 'Arabic',
 'caesar cipher',
 'French',
 'Adyghe',
 'Morse code',
 'Python',
 'reverse fibonacci indices in each word',
 'swap even odd letters in each word',
 'base64 3x cipher',
 'base64 2x cipher',
 'remove all nouns',
 'enterprise Java',
 'gzip to base64 encoded'
 
]

l_fixed_encoding_ordering = [s.replace(' ', '_') for s in l_fixed_encoding_ordering]

In [ ]:
# --- Compute PGR (percent of identity performance) ---
df_pgr = df_viz_tmp.copy()

df_pgr['model'] = df_pgr['model'].str.split('Qwen/').str[-1]

df_pgr = df_pgr[~df_pgr['model'].str.contains('Qwen')]

# Identity baseline per model
identity_baseline = (
    df_pgr[df_pgr["encoding_scheme"] == "identity"]
    .set_index("model")["adherent_and_correct"]
)

df_pgr["identity_value"] = np.maximum(df_pgr["model"].map(identity_baseline), 0.0001)

# Avoid divide-by-zero
# df_pgr = df_pgr[df_pgr["identity_value"] > 0].copy()

# Mean PGR as percentage
df_pgr["PGR_pct"] = (df_pgr["adherent_and_correct"] / df_pgr["identity_value"])

# CI deltas -> percentage deltas relative to identity baseline
# low is negative, high is positive
df_pgr["PGR_pct_hi_ci"]  = (df_pgr["adherent_and_correct_hi_ci"] / df_pgr["identity_value"])
df_pgr["PGR_pct_low_ci"] = (df_pgr["adherent_and_correct_low_ci"]        / df_pgr["identity_value"])

df_pgr.loc[df_pgr["encoding_scheme"] == "identity", ["PGR_pct_hi_ci", "PGR_pct_low_ci"]] = 0.0

# order by sonnet 4 hard coded
# df_sonnet4 = df_pgr[df_pgr['model'].str.contains('claude-3-5-sonnet')].sort_values('adherent_and_correct', ascending=False)
# df_sonnet4 = df_pgr[df_pgr['model'].str.contains('14B')].sort_values('adherent_and_correct', ascending=False)
df_sonnet4 = df_pgr[df_pgr['model'].str.contains('sonnet-4')].sort_values('backtranslation_bleu_scores', ascending=False)

d_encoding_scheme_to_idx = {}
for i in range(len(df_sonnet4)):
    d_encoding_scheme_to_idx[df_sonnet4['encoding_scheme'].iloc[i]] = i

# d_encoding_scheme_to_idx['identity'] = 0

# for i, encoding in enumerate(l_fixed_encoding_ordering):
#     d_encoding_scheme_to_idx[encoding] = i

df_pgr = df_pgr[~df_pgr['encoding_scheme'].isin(l_themed_encodings)]
df_pgr = df_pgr[~df_pgr['encoding_scheme'].isin(l_ignore_languages)]

df_pgr['sort_order'] = df_pgr['encoding_scheme'].map(d_encoding_scheme_to_idx)
df_pgr = df_pgr.sort_values(['sort_order'], ascending=True)

s_encoding_scheme_order = df_sonnet4['encoding_scheme']

df_pgr['encoding_scheme'] = df_pgr['encoding_scheme'].map(lambda s: " ".join(s.split('_')))

df_pgr['Model Family'] = df_pgr["model"].map(lambda x: "Qwen2.5" if "Qwen2.5" in x else "GPT" if "gpt" in x else "Claude")

In [ ]:
fig = px.line(df_pgr[df_pgr['encoding_scheme'] != 'zero shot'],
              x='encoding_scheme',
              y='backtranslation_bleu_scores',
              color='model',
              color_discrete_map={**d_claude_gradient, **d_gpt_gradient, **d_qwen_gradient},
              markers=True,
              # facet_col="Model Family"
)

fig.update_layout(
    # title=dict(text=title, font=dict(size=22)),
    template="plotly_white",
    height=500,
    width=1800,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.08,
        xanchor="left",
        x=0.01,
        font=dict(size=15)
    ),
    margin=dict(t=0, r=0, b=0, l=0),
    font=dict(size=15),
    hovermode='x unified'
)

fig.update_xaxes(title="Cipher")
fig.update_yaxes(range=[0, 101], dtick=10, title="BLEU score")

fig.add_hrect(
    y0=60, y1=100,
    fillcolor="#D3D3D3", opacity=0.4,
    layer="below", line_width=0,
    annotation_text="Fluent decoding",
    annotation_font=dict(color="grey")
)

# fig.update_layout(
#     xaxis=dict(
#         categoryorder="array",
#         categoryarray=s_encoding_scheme_order
#     )
# )

fig.update_layout(width=600, height=600, title=None)
fig.update_xaxes(tickangle=90)
fig.update_layout(height=510, showlegend=True)

fig.show('png')


In [ ]:
df_pgr_save = df_pgr[['data', 'backtranslation_bleu_scores', 'encoding_scheme', 'model', 'input_type']]

df_pgr_save.to_parquet("zero_shot_decode_data.parquet")

In [ ]:
df_pgr_save[df_pgr_save['model'] == 'gpt-4.1-2025-04-14'].drop_duplicates(subset=['model', 'encoding_scheme'])

In [ ]:
df_pgr[(df_pgr['model'] == 'claude-sonnet-4-20250514') & (df_pgr['encoding_scheme'] == 'base64 2x cipher')]

In [ ]:
df = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/63fec9362cf90b227d87cbade5c65dab5003e0fb/data/joined_output.parquet")

df.head()

In [ ]:
idx = 124

In [ ]:
print(df['translated_solution'].iloc[idx])

In [ ]:
print(df['generated_backtranslations'].iloc[idx][0])

In [ ]:
df['backtranslation_bleu_scores'].iloc[idx][0]